<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/115_%E4%BE%8B%E5%A4%96%E3%81%A8%E3%82%B3%E3%83%B3%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%83%9E%E3%83%8D%E3%83%BC%E3%82%B8%E3%83%A3%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

例外とコンテキストマネージャー
==============================

エラーと例外
------------

### 構文エラーと実行時例外 ###

エラーと例外の基本については、公式チュートリアルの「[エラーと例外](https://docs.python.org/ja/3/tutorial/errors.html)」でほぼ十分。

Python のインタープリターがコードを処理する際には、以下の処理が行われる。

  1. **字句解析**（lexical analysys）:  
ソースコードのテキストを読み込み、それらを Python 言語内で使われる単語の列、すなわちトークン列へ変換する。
  2. **構文解析**（syntax analysys）:  
トークン列から抽象構文木（Astract Syntax Tree; AST）を生成する。
  3. **コード生成**（code generation）:  
AST オブジェクトからコードオブジェクトを生成する。
  4. **コンパイル**（compilation）:  
コードオブジェクトから実行可能なバイトコードを生成する。
  5. **実行**（execution）:  
バイトコードを実行する。

コンパイルまでの段階で送出されたエラーが**構文エラー**（syntax error）であり、バイトコードの実行時に送出されたエラーが**実行時例外**（runtime exception）である。

Python でこれらは全て**例外**（exception）として扱われるように組み込みクラスが用意され、統一的に処理できるようになっている。

### try 文 ###

Python で例外処理は try 文で行われる。try 文の基本的な構文は以下の通り。

``` python
try:
    エラーを捕捉したいコード

except [例外 [as 変数]]:
    try 節で例外（や派生クラスの例外）が発生した場合に実行されるコード（＝例外ハンドラ）

（他の except 節）

else:
    try 節で全く例外が送出されなかったときに実行されるコード

finally:
    try 節での例外発生の有無に関わらず必ず実行されるコード（＝クリーンアップ）
```

try 文では、必ず 1 個以上の except 節か、または、1 つの finally 節がなければならない。except 節も finally 節もない文は許されない。

except 節を複数置く場合、まず一番上の except 節で指定された例外と派生クラスの例外のいずれか 1 つが try 節で発生した例外とマッチするかどうかテストされる。マッチしたときは対応するハンドラが実行されて、下に置かれた except 節が全て無視される。マッチしなかったときは次の except 節で同様のテストが行われ、以降同様となる。したがって、基底クラスを指定する except 節を置くと、その下に置かれた派生クラスを指定する except 節は常に無視される。

except 節ではタプルで複数の例外を指定することもできる:

``` python
except (例外, 例外, ...)[as 変数]:
    例外ハンドラ
```

else 節は完全にオプションである。else 節は全ての except 節よりも下に置かなければならない。

finally 節は存在する全ての except 節と else 節よりも下に置かなければならない。finally 節のブロックは、try 節、except 節、else 節で処理されない例外があればその送出前に必ず実行される。

finally 節を置くときは、except 節と else 節を省略できるので、次がもっとも簡単な形式である。

``` python
try:
    コード

finally:
    try 節での例外発生の有無に関わらず必ず実行されるコード（＝クリーンアップ）
```

この形式の try 文は、例外処理ではなくてクリーンアップ動作を定義するためだけに使用される。

try 文において、try 節、except 節、else 節、finally 節の各ブロックではスコープは形成されない（if 文や for 文などと同様である）。

In [1]:
x = 100
try:
    x = 1
finally:
    x += 1
print(x)

2


例外が `as 変数` を使って代入されたとき、それは except 節の終わりに消去される。これはちょうど、以下のコード:

``` python
except E as N:
    foo
```

が、以下のコードに翻訳されたかのようなものである:

``` python
except E as N:
    try:
        foo
    finally:
        del N
```

したがって、例外を except 節以降で参照できるようにするためには、別の変数に代入しておく必要がある。

※ [公式ドキュメント](https://docs.python.org/ja/3/reference/compound_stmts.html#except-clause)によれば、例外変数が削除される理由について次のように説明してる。

> 例外が削除されるのは、トレースバックが付与されると、そのスタックフレームと循環参照を形作り、次のガベージ収集までそのフレーム内のすべての局所変数を生存させてしまうからです。

### トレースバック ###

プログラムの実行中にエラーが発生したときに、そのエラーがどのような経緯で発生したかを示すレポートために、コールスタックが関数の履歴という形で利用される。このレポートを**スタックトレース**（stack trace）という。

Python のインタープリターは、例外が try-except 文で処理されなかった場合に、任意のエラーメッセージを含んだスタックトレースを標準エラー出力に出力する。これを**トレースバック**（traceback）と呼ぶ。トレースバックは「Traceback (most recent call last):」の行で始まる。

``` text
>>> '2' + 2
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: can only concatenate str (not "int") to str
```

例外クラス
----------

### 基底クラス ###

`BaseException` クラスは、全ての組み込み例外の基底クラスであり、例外処理時に利用される基本的な属性やメソッドをサポートする。

`BaseException` の主な属性:

| 属性 | 意味 |
|:---|:---|
| `args` | 例外コンストラクタに与えられた引数のタプルが格納されている |

`BaseException` の主なメソッド:

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `add_note(note)` | Python 3.11 で追加。特殊属性 `__notes__` が保持する文字列のリスト（ノート）に文字列 `note` を追加する。`note` 引数が<br />文字列以外の場合 `TypeError` 例外が送出される | `None` |

ノートは標準のトレースバックで例外文字列の後に表示される。

`BaseException` は基本的には try-except 文で捕捉すべきではない。except 節は派生クラスのインスタンスである例外ともマッチするので、`BaseException` を指定した except 節では try-except 文で処理すべきではない例外まで捕捉してしまうからである。そのような例外は直ちにインタープリターを終了させるべきものであり、「システム終了に関する例外」と呼ばれる。

例外を指定しない except 節は、`BaseException` の指定と同様に扱われる。このため、PEP 8 は例外を指定しない except 節ではなく、特定の例外を指定することを推奨している。ただし、例外ハンドラがトレースバックを出力するかログを出力する場合には、裸の except 節も許容されるとしている。

`BaseException` を直接継承する `Exception` クラスは、システム終了以外のすべての組み込み例外の基底クラスとなっている。`Exception` の派生クラスには、実行時例外だけでなく構文エラーを表現するクラスも含まれることに注意する。

`BaseException` を直接継承する方法は、`Exception` を除いて「システム終了に関する例外」向けに予約される。このため、PEP 8 では、ユーザ定義例外は `BaseException` ではなくて `Exception` から派生させるべきであるとされる。

### システム終了に関する例外クラス ###

以下は `BaseException` を直接継承するクラスであり、「システム終了に関する例外」となるもので、基本的には try-except 文で処理するのではなくて直ちにインタープリターを終了させるべきものである。

| 例外クラス | 意味 |
|:---|:---|
| `GeneratorExit` | ジェネレータやコルーチンが閉じられたときに送出される例外 |
| `KeyboardInterrupt` | ユーザが割り込みキーを押した場合に送出される例外 |
| `SystemExit` | `sys.exit()` 関数から送出される例外 |

`sys.exit([arg])` は、`SystemExit` 例外を送出して、Python インタープリターを終了する。引数 `arg` に終了コードを指定できる。数値以外のオブジェクトを渡した場合は、渡されたオブジェクトを文字列として `sys.stderr` （標準エラー出力）に出力し、呼び出し元に終了コード 1 を返して終了する。また、引数を省略した場合は、終了コード 0 として終了する。一般に、終了コード 0 は正常終了、それ以外は異常終了を表す。

try 文のブロック内に `sys.exit()` を書き finally 節を追加すると、Python インタープリターを終了する前に finally 節のブロックが実行される。作業用ファイルの削除・リソースの解放など必ず行わなければならない処理は、try/finally を使って実行されるようにする必要がある。

``` python
import sys
try:
    sys.exit('sys.exit() is called')
finally:
    # 終了前に実行されるコード
```

### システム終了以外の例外クラス ###

`ArithmeticError` は、算術上のエラーに対して送出される。派生クラスに、`OverflowError`（オーバーフロー時に送出）・`ZeroDivisionError`（0除算時に送出）。

`AttributeError` は、存在しない属性を参照したときに送出される。

`ImportError` は、import 文でモジュールをロードしようとして問題が発生したときに送出される。派生クラスに、`ModuleNotFoundError`（モジュールが存在しないときに送出）。

`LookupError` は、存在しないインデックスやキーで参照した場合に送出される。派生クラスに、`IndexError`（インデックス無効時に送出）・`KeyError`（キー無効時に送出）。

`NameError` は、存在しない関数を呼び出したり、存在しない変数を参照したときに送出される。派生クラスに、`UnboundLocalError`（値なしローカル変数参照時に送出）。

In [ ]:
def hoge():
    print(x)  # 未定義の変数を参照しようとしている
    x = 1

try:
    hoge()
except Exception as err:
    print(f"{type(err).__name__}: {err}")

UnboundLocalError: local variable 'x' referenced before assignment


`OSError` は、システム関数がシステム関連のエラーを返した場合に送出される。派生クラスに、`FileExistsError`（すでに存在するファイル・ディレクトリを作成しようとするときに送出）・`FileNotFoundError`（ファイル・ディレクトリが存在しないときに送出）など。

`TypeError` は、関数や演算を適切でない型のオブジェクトに適用した場合に送出される。

`ValueError` は、関数や演算を適切でない値に適用した場合に送出される。派生クラスに、`UnicodeError`（Unicode に関するエンコードまたはデコードのエラーが発生した際に送出）。

`RuntimeError` は、他のカテゴリに分類できない実行時例外が検出された場合に送出される。

`SyntaxError` は、構文エラーの基底クラスである。派生クラスに、`IndentationError`（インデントが正しくない場合に送出）。また、`IndentationError` の派生クラスに `TabError`（タブとスペースを一貫しない方法でインデントに使っている場合に送出）。

`SyntaxError` も `Exception` の派生クラスになっているが、**これは通常 try-except で捕捉することはできない**。構文エラーはバイトコードが実行される前に発生し、その時点では try-except ブロック自体が評価されていないため、エラーを捕捉することができないのである。

そうだとすると `SyntaxError` を `Exception` の派生クラスとする意味がないようにみえるが、以下のような特殊なケースでは構文エラーが捕捉される。

  * 組み込みの `eval()` 関数や `exec()` 関数を使用して動的にコードを実行する場合

このケースでは、関数の実行時に `SyntaxError` が発生する可能性があるため、try-except で捕捉することが可能である。

In [ ]:
try:
    exec("print('Hello'")  # 閉じ括弧が足りないため SyntaxError が発生する
except Exception as err:
    print(f"{type(err).__name__}: {err}")

SyntaxError: '(' was never closed (<string>, line 1)


### 例外の連鎖 ###

**例外の連鎖**（exception chaining）は、ある例外の処理中に別の例外が発生した場合に、それらを関連付けてトレースできるようにする仕組みである。具体的には、以下の特殊属性によって構成される。

  * `BaseException.__context__`
  * `BaseException.__cause__`
  * `BaseException.__suppress_context__`

例外インスタンスの作成時には `__context__` も `__cause__` も `None` で初期化され、`__suppress_context__` は `False` で初期化される。

In [ ]:
try:
    raise OSError("foo")
except OSError as e:
    print(f"e is {type(e).__name__} ---")
    print(f"{e.__context__ = }")
    print(f"{e.__cause__ = }")
    print(f"{e.__suppress_context__ = }")

e is OSError ---
e.__context__ = None
e.__cause__ = None
e.__suppress_context__ = False


ある例外が except 節ですでに処理されているときに新しい例外を発生させると、新しい例外の `__context__` 属性は、処理された例外に自動的にセットされる。この形は**暗黙的な連鎖**となる。

In [ ]:
def foo():
    raise OSError("foo")

def bar():
    try:
        foo()
    except OSError:
        raise RuntimeError

try:
    bar()
except RuntimeError as e:
    # 暗黙的な連鎖
    print(f"e is {type(e).__name__} ---")
    print(f"{e.__context__ = }")
    print(f"{e.__cause__ = }")
    print(f"{e.__suppress_context__ = }")

e is RuntimeError ---
e.__context__ = OSError('foo')
e.__cause__ = None
e.__suppress_context__ = False


raise 文が from を伴う場合、つまり

``` python
raise new_exc from original_exc
```

の形（式 `original_exc` は例外か `None` でなければならない）の場合、`new_exc` の `__cause__` 属性の値が `original_exc` にセットされ、同時に `__suppress_context__` 属性の値が `True` にセットされる。この形は**明示的な連鎖**となる。

In [ ]:
def foo():
    raise OSError("foo")

def bar():
    try:
        foo()
    except OSError as e:
        raise RuntimeError from e

try:
    bar()
except RuntimeError as e:
    # 明示的な連鎖
    print(f"e is {type(e).__name__} ---")
    print(f"{e.__context__ = }")
    print(f"{e.__cause__ = }")
    print(f"{e.__suppress_context__ = }")

e is RuntimeError ---
e.__context__ = OSError('foo')
e.__cause__ = OSError('foo')
e.__suppress_context__ = True


例外の連鎖は、例外が発生した経緯に関する情報を提供する。例外のトレースバック表示では、その例外自体のトレースバックに加え、連鎖された例外を表示する。

暗黙的な連鎖の場合:

``` python
# content of sample.py
def foo():
    raise OSError("foo")

def bar():
    try:
        foo()
    except OSError:
        raise RuntimeError

bar()
```

``` text
# content of sample.py
Traceback (most recent call last):
  File "sample.py", line 7, in bar
    foo()
  File "sample.py", line 3, in foo
    raise OSError("foo")
OSError: foo

During handling of the above exception, another exception occurred:  （訳: 上記の例外の処理中に別の例外が発生した:）

Traceback (most recent call last):
  File "sample.py", line 11, in <module>
    bar()
  File "sample.py", line 9, in bar
    raise RuntimeError
RuntimeError
```

明示的な連鎖の場合:

``` python
# content of sample.py
def foo():
    raise OSError("foo")

def bar():
    try:
        foo()
    except OSError as e:
        raise RuntimeError from e

bar()
```

``` text
$ python sample.py
Traceback (most recent call last):
  File "sample.py", line 7, in bar
    foo()
  File "sample.py", line 3, in foo
    raise OSError("foo")
OSError: foo

The above exception was the direct cause of the following exception:  （訳: 上記の例外は、次の例外の直接の原因だった）

Traceback (most recent call last):
  File "sample.py", line 11, in <module>
    bar()
  File "sample.py", line 9, in bar
    raise RuntimeError from e
RuntimeError
```

明示的な連鎖は、例外を明示的に変換する意図で使われる。明示的に連鎖させた例外は（存在すれば）常に表示される。これに対して、暗黙に連鎖させた例外は `__cause__` が `None` かつ `__suppress_context__` が `False` の場合にのみ表示される。`raise new_exc from None` のようにすると、`new_exc` に対して暗黙に連鎖させた例外は表示されなくなる。

``` python
# content of sample.py
def foo():
    raise OSError("foo")

def bar():
    try:
        foo()
    except OSError as e:
        raise RuntimeError from None

bar()
```

``` text
$ python sample.py
Traceback (most recent call last):
  File "sample.py", line 11, in <module>
    bar()
  File "sample.py", line 9, in bar
    raise RuntimeError from None
RuntimeError
```

`raise new_exc from None` の形は、例外を変換するときに古い例外を非表示にするために使われる。このようにしても、`__context__` には古い例外の情報が残っているので、古い例外の調査は可能である。

### 例外の参照 ###

`sys.exception()` 関数（Python 3.11 で追加）は、 except 句のブロック（例外ハンドラ）で呼び出されると、その例外ハンドラで捕捉されている例外を返す。例外ハンドラが互いにネストされている場合、最も内側のハンドラによって処理された例外にのみアクセスできる。例外ハンドラが実行されていない場合、この関数は `None` を返す。

``` python
>>> import sys
>>> try:
...     raise TypeError
... except:
...     print(repr(sys.exception()))
...     try:
...          raise ValueError
...     except:
...         print(repr(sys.exception()))
...     print(repr(sys.exception()))
...
TypeError()
ValueError()
TypeError()
>>> print(sys.exception())
None
```

Python 3.10 以前では、 `sys.exc_info()` 関数が使用できる（もちろん Python 3.11 以降でも使用できる）。この関数は、例外 `e` が現在処理されている場合に、タプル `(type(e), e, e.__traceback__)` を返す。例外の `__traceback__` 属性には、例外が最後に発生した時点での呼び出しスタックをカプセル化するトレースバックオブジェクトが格納されている。スタック上のどこにも例外が処理されていない場合、この関数は 3 つの `None` 値を含むタプルを返す。

In [ ]:
import sys
try:
    raise ValueError
except:
    exc_type, exc, exc_traceback = sys.exc_info()
print(f"{exc_type=}")
print(f"{exc=}")
print(f"{exc_traceback=}")

exc_type=<class 'ValueError'>
exc=ValueError()
exc_traceback=<traceback object at 0x7ca471264f80>


コンテキストマネージャー
------------------------

### コンテキストマネージャーと with 文 ###

**コンテキストマネージャー**（context manager）とは、次の二つのメソッドをサポートするオブジェクトである。

  * `__enter__()`:
  * `__exit__(exc_type, exc_val, exc_tb)`:

コンテキストマネージャーと with 文は、try/finally 文の標準的な使い方をコードから切り出すことを目的として導入された。

with 文の構文は次のとおり。

``` python
with 式[ as 変数[, 式[ as 変数[, ...]]]:
    ...
```

Python 3.10 からは、`with` の後を丸括弧で囲めるようになった（タプルとはされなくなった）。丸括弧の内側は改行ができる。

``` python
with (式[ as 変数],
      式[ as 変数],
      ...,
      式[ as 変数]):
    ...
```

with 文の式は、評価結果がコンテキストマネージャーであることが要求される（以下、**コンテキスト式**と呼ぶ）。

一つのコンテキスト式を持つ with 文の実行は以下のように進行する。

  1. コンテキスト式を評価することで、コンテキストマネージャーを取得する。
  2. コンテキストマネージャーの `__enter__()` メソッドが、後で使うためにロードされる。
  3. コンテキストマネージャーの `__exit__()` メソッドが、後で使うためにロードされる。
  4. コンテキストマネージャーの `__enter__()` メソッドを呼び出す。
  5. with 文に `as 変数` があれば、変数に `__enter__()` が返す値を代入する。
  6. with 文の本体を実行する。
  7. with 文の本体で例外が発生したか否かで場合分けされる。
      * 例外が発生した場合、例外の型、例外オブジェクト、トレースバックをこの順で位置引数として渡して `__exit__()` を呼び出す。`__exit__()` からの戻り値が偽ならば、例外が再送出される。この戻り値が真ならば例外は抑制され、 with 文のブロックから抜ける。
      * 例外が発生しなかった場合、3 つの `None` を位置引数として渡して `__exit__()` を呼び出す。 `__exit__()` の戻り値は無視され、 with 文のブロックから抜ける。

次のコード:

``` python
with EXPRESSION as TARGET:
    SUITE
```

これは次と等価である:

``` python
manager = (EXPRESSION)
enter = type(manager).__enter__
exit = type(manager).__exit__
value = enter(manager)
hit_except = False

try:
    TARGET = value
    SUITE
except:
    hit_except = True
    if not exit(manager, *sys.exc_info()):
        raise
finally:
    if not hit_except:
        exit(manager, None, None, None)
```

try-except 文のブロックではスコープは形成されないのと同様、with 文のブロックではスコープは形成されない。

In [ ]:
class MyManager:
    def __enter__(self):
        print("enter")
        return "Hello, World!"

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("exit")


with MyManager() as greeting:
    msg = greeting
    print(msg)

print(msg)  # with文のブロック内の変数は外でも参照できる

print("-" * 30)

try:
    with MyManager() as greeting:
        raise RuntimeError("with 本体でエラー")
except RuntimeError as err:
    print(f"{type(err).__name__}: {err}")

enter
Hello, World!
exit
Hello, World!
------------------------------
enter
exit
RuntimeError: with 本体でエラー


複数のコンテキスト式があるとき、コンテキストマネージャーは複数の with 文がネストされたかのように進行する:

``` python
with A() as a, B() as b:
    SUITE
```

これは次と等価である:

``` python
with A() as a:
    with B() as b:
        SUITE
```

### contextlib.contextmanager ###

標準ライブラリの `contextlib` モジュールが提供する `contextmanager()` は、コンテキストマネージャーの生成に便利なデコレーターである。`__enter__()` と `__exit__()` メソッドをもつクラスを定義する代わりに、このデコレーターを付けたジェネレーター関数によってコンテキストマネージャーを生成することができる。次のコードで、`contextlib.contextmanager()` の使い方を示す:

In [ ]:
from contextlib import contextmanager

@contextmanager
def mymanager():
    print('enter')
    try:
        yield "Hello, World!"
    finally:
        print('exit')

with mymanager() as greeting:
    print(greeting)

enter
Hello, World!
exit


yield 文より上の処理は、with 文の本体より前に実行される。`as 変数` に渡したい値があるときは、yield に値を渡す。yield 文より下の処理は、with 文の本体の実行後に実行される。注意点は、yield 文より下の処理が最後に必ず実行されるわけではないことである。with 文の本体の実行中にエラーが発生すると、yield 文を実行している箇所で再送出されるので、finally 節を使わない場合、yield 文より下の処理は実行されない。上のコードで `finally:` の行を削除した形:

``` python
from contextlib import contextmanager

@contextmanager
def mymanager():
    print('enter')
    yield "Hello, World!"
    print('exit')  # エラーが発生すると実行されない

with mymanager() as greeting:
    print(greeting)
    raise Exception
```

このコードを実行すると、以下のように出力が行われて、`print('exit')` が実行されないことがわかる。

``` text
Hello, World!
Traceback (most recent call last):
  File ..., line 11, in <module>
    raise Exception
Exception
```

### contextlib.closing ###

一般に、モジュールが提供するクラスが `close()` メソッドをサポートする場合、 `close()` メソッドはオブジェクトを閉じる終了処理を実装する。クラスがコンテキストマネージャーとして使用できる場合、通常 with 文のブロックの完了時に `close()` メソッドを呼び出す。クラスがコンテキストマネージャーとして使用できない場合に `close()` メソッドを確実に呼び出したいなら、 `contextlib.closing()` 関数を使用するとよい。

`contextlib.closing()` 関数は、渡されたオブジェクトの `close()` メソッドをブロックの完了時に呼び出すコンテキストマネージャーを返す。これは基本的に以下と等価である:

``` python
from contextlib import contextmanager

@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()
```

使用例:

In [ ]:
from contextlib import closing

class MyClass:
    def say(self):
        print("hello")

    def close(self):
        print("goodby")

with closing(MyClass()) as m:
    m.say()

hello
goodby


### contextlib.suppress ###

`contextlib.suppress()` は、with 文の内部で指定された例外の発生を抑えるコンテキストマネージャーを返す。

In [ ]:
from contextlib import suppress

with suppress(FileNotFoundError, NameError):
    raise FileNotFoundError
    raise NameError

これは以下と等価である:

In [ ]:
try:
    raise FileNotFoundError
except (FileNotFoundError, NameError):
    pass

警告
----

### 基底クラス ###

`Exception` クラスのサブクラス `Warning` は、警告クラスの基底クラスである。警告は、一般の例外とは異なる以下の特徴を持つ。

  * 警告が発せられても、プログラムの実行は停止されない。
  * 発せられた警告をそのままでは try-except 文で捕捉できない。

警告メッセージは通常 `sys.stderr` に出力されるが、すべての警告を無視したり、警告を例外にしたりと、その処理を柔軟に変更することができる。警告の処理は、警告カテゴリ（警告クラス）、警告メッセージのテキスト、警告が発行されたソースの位置に基づいて変化する。同じソースの位置で特定の警告が繰り返された場合、通常は 2 回目以降の出力が抑制される。

### 警告フィルター ###



警告の処理は、**警告フィルター**によって制御される。警告フィルターはリスト構造になっていて、各エントリは警告の処理方法と警告を特定するための 4 条件を表すタプル `(action, message, category, module, lineno)` である。各フィールドの意味は次のとおり。

| フィールド | 意味 |
|:---|:---|
| `action` | 残りのフィールドと一致する警告に対して処理する内容を指定する文字列<br /><br />・`'default'`: ソース位置ごとに、一致した警告のうち最初の警告のみ出力する<br /><br />・`'module'`: モジュールごとに、一致した警告のうち最初の警告のみ出力する<br /><br />・`'once'`: 一致した警告のうち最初の警告のみ出力する<br /><br />・`'always'`: 一致した警告を何回でも出力する<br /><br />・`'ignore'`: 一致した警告を出力しない<br /><br />・`'error'`: 一致した警告を一般の例外（エラー）に変換する
| `message` | 警告メッセージ全体と一致させる文字列。この一致では大文字と小文字は区別されない。空の文字列は全ての警告メッセージと一致させる |
| `category` | 警告クラスまたはその基底クラスの名前と一致させる文字列 |
| `module` | 警告が発せられたソースのモジュール名と一致させる文字列。この一致では大文字と小文字が区別される。空の文字列は全てのモジュール名と一致させる |
| `lineno` | 警告が発せられたソースの行番号と一致させる整数。`0` はすべての行番号と一致させる |

警告フィルター内の各エントリの順番は優先順位の高い順になる。つまり、複数のエントリが特定の警告に一致した場合、リストの先頭に近いほうのエントリが後方にあるエントリに優先する。

Python インタープリターに与える `-W` フラグまたは環境変数 `PYTHONWARNINGS` で警告フィルターのエントリを設定することができる。`-W` 引数の完全形は次のようになる:

``` shell
python -W action:message:category:module:lineno script.py
```

`-W` と `action` の間の空白は無くてもよい。`action` 以外は省略できる。`message` や `module` を省略した場合は、空の文字列を指定したのと同等である。`category` を省略した場合は、`Warning` を指定したのと同等である。`lineno` を省略した場合は、`0` を指定したのと同等である。後ろのフィールドが全て省略されているときのコロン `:` は省略できる。したがって、最も簡単な形は `-Waction` となる。たとえば、`-Wignore`。これはさらに短縮形 `-Wi` が使える。他の `action` についても同様である。

`'default'` の処理は、通常の警告の処理である。したがって、単に通常では出力されない警告を出力するだけなら、`-Wdefault` または `-Wd` 引数を指定することで実現できる。これは、`-Wdefault::Warning::0` と同等である。

複数の `-W` 引数を指定することができる。左の引数から順に警告フィルターの先頭にエントリが挿入される。したがって、複数の `-W` 引数では、最後にマッチした引数の `action` が有効になる。

環境変数 `PYTHONWARNINGS` も `-W` フラグと同等で、カンマ区切りの文字列に設定すると、`-W` を複数回指定するのと同じになり、リストの後のフィルタがリストの前のフィルタよりも優先される。

### 警告クラス ###

全ての警告クラスは、`Warning` クラスのサブクラスとしなければならない。主な組み込みの警告は次のとおり。

  * `DeprecationWarning` : 廃止予定の Python 機能についての警告
  * `SyntaxWarning`: 曖昧な構文に対する警告
  * `FutureWarning`: Python で書かれたアプリケーションにおいて非推奨とされた機能についての警告
  * `RuntimeWarning`: 実行時に発生した問題についての警告
  * `UserWarning`: ユーザーのコードによって生成される警告
  * `ResourceWarning`: リソースの使用に関連した警告。通常では、この警告メッセージは出力されない。

たとえば、文字列リテラルや数値リテラルを `is` で比較すると、`SyntaxWarning` が発せられる（Colab では警告メッセージが 3 重に出力される）。

In [ ]:
i = 0
for _ in range(5):
    # 数値リテラルを is で比較している
    if 100 is 100: pass

<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-7-4b3dbc1cd103>:4: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if 100 is 100: pass


一般の例外が発生した時に、何が間違っているのかを説明するために `SyntaxWarning` が同時に発せられることもある。

In [ ]:
try:
    # コンマの入れ忘れ
    data = [(1, 2, 3)(4, 5, 6)]
except TypeError as err:
    print(f"{type(err).__name__}: {err}")

TypeError: 'tuple' object is not callable


<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<ipython-input-8-c58b4b424a22>:3: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  data = [(1, 2, 3)(4, 5, 6)]


全ての警告クラスの基底である `Warning` クラスが `Exception` クラスから派生しているので、`raise Warning(message)` などと raise 文で警告クラスを使用できるが、これは警告をエラーに変換する。エラーに変換された後は、一般の例外処理と同様に、try-except 文で処理することができる。

### warnings モジュール ###


手動で警告を発するには、標準ライブラリの `warnings` モジュールが提供する `warn()` 関数を使う。

``` python
warnings.warn(message, category=None, stacklevel=1, source=None, *, skip_file_prefixes=None)
```

| 引数 | 意味 |
|:---|:---|
| `message` | 警告メッセージ |
| `category` | 警告クラス。指定を省略するか、`None` を指定すると、`UserWarning` が使われる |
| `stacklevel` | 警告を発する位置を、`warn()` 関数の内部から見て何回呼び出しをさかのぼった位置にするかを指定する。`1` の場合、`warn()` の呼び出し位置について警告を発する。<br />`2` の場合、`warn()` を使用する関数の呼び出し位置について警告を発する |
| `source` | `category` に `ResourceWarning` を指定する場合に、`source` オプションにその警告の原因となったオブジェクトを指定できる |
| `skip_file_prefixes` | Python 3.12 で追加されたキーワード専用引数。文字列のタプルを指定すると、それらを `stacklevel` に関するカウントをスキップするソースの名前の接頭辞とみなす<br />例: `skip_file_prefixes=(os.path.dirname(__file__),)` |

次のコードでは、 `obsolete_func()` はアプリケーションにおいて非推奨とされた関数と仮定している。これを呼び出す場合に、その関数内で警告を発するのではなく、呼び出し元で警告を発するようにしている。

In [ ]:
import warnings

def obsolete_func():
    warnings.warn("この関数は廃止予定です", FutureWarning, stacklevel=2)

obsolete_func()  # この位置で警告が発せられる

<ipython-input-9-1081ca4fffcb>:6: FutureWarning: この関数は廃止予定です
  obsolete_func()  # この位置で警告が発せられる


Python コード内で警告フィルターにエントリを追加することができる。`warnings` モジュールが提供する以下の関数を使う。

``` python
warnings.filterwarnings(action, message='', category=Warning, module='', lineno=0, append=False)
```

この関数は、警告フィルターの先頭にエントリ `(action, message, category, module, lineno)` を挿入する。`append` が `True` の場合は、末尾に挿入する。`message` および `module` には正規表現を使用できる（これはコマンドライン上では実現できない機能である）。

``` python
warnings.simplefilter(action, category=Warning, lineno=0, append=False)
```

この関数は、警告フィルターの先頭にエントリ `(action, '', category, '', lineno)` を挿入する。`append` が `True` の場合は、末尾に挿入する。

``` python
warnings.resetwarnings()
```

この関数は、警告フィルターをリセットする。

たとえば、`warnings.simplefilter('ignore')` を呼び出すと、全ての警告と一致する `'ignore'` アクションのエントリが警告フィルターの先頭に挿入され、結果、以降は全ての警告が出力されなくなる。

In [ ]:
import warnings

# すべての警告が非表示になる
warnings.simplefilter("ignore")

warnings.warn("This is a warning", UserWarning)

# UserWarning 警告を例外として扱う
warnings.simplefilter("error", UserWarning)

try:
    warnings.warn("This is a warning", UserWarning)
except UserWarning:
    print("Caught the warning as an exception")

# 警告フィルターをリセットする
warnings.resetwarnings()

Caught the warning as an exception


`warnings.simplefilter()` 関数などを使って警告フィルターを変更すると、その影響はアプリケーション全体に及ぶ。`warnings.resetwarnings()` 関数は警告フィルターを完全にリセットしてしまう。

一時的に警告を制御したいだけの場合には、`warnings.catch_warnings` コンテキストマネージャーを使用する。これは、警告フィルターをコピーし、終了時に警告フィルターを復元するコンテキストマネージャーである。

In [ ]:
import warnings

def obsolete_func():
    warnings.warn("この関数は廃止予定です", FutureWarning, stacklevel=2)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    obsolete_func()

print("with 文を抜けた")
warnings.warn("something")

with 文を抜けた


<ipython-input-16-6f8c25771269>:11: UserWarning: something
  warnings.warn("something")


組み込み例外のクラス階層
------------------------

``` text
BaseException
 ├── BaseExceptionGroup
 ├── GeneratorExit
 ├── KeyboardInterrupt
 ├── SystemExit
 └── Exception
         ├── ArithmeticError
         │      ├── OverflowError
         │      └── ZeroDivisionError
         ├── AssertionError
         ├── AttributeError
         ├── BufferError
         ├── EOFError
         ├── ExceptionGroup [BaseExceptionGroup]
         ├── ImportError
         │      └── ModuleNotFoundError
         ├── LookupError
         │      ├── IndexError
         │      └── KeyError
         ├── MemoryError
         ├── NameError
         │      └── UnboundLocalError
         ├── OSError
         │      ├── BlockingIOError
         │      ├── ChildProcessError
         │      ├── ConnectionError
         │      │      ├── BrokenPipeError
         │      │      ├── ConnectionAbortedError
         │      │      ├── ConnectionRefusedError
         │      │      └── ConnectionResetError
         │      ├── FileExistsError
         │      ├── FileNotFoundError
         │      ├── InterruptedError
         │      ├── IsADirectoryError
         │      ├── NotADirectoryError
         │      ├── PermissionError
         │      ├── ProcessLookupError
         │      └── TimeoutError
         ├── ReferenceError
         ├── RuntimeError
         │      ├── NotImplementedError
         │      └── RecursionError
         ├── StopAsyncIteration
         ├── StopIteration
         ├── SyntaxError
         │      └── IndentationError
         │              └── TabError
         ├── SystemError
         ├── TypeError
         ├── ValueError
         │      └── UnicodeError
         │              ├── UnicodeDecodeError
         │              ├── UnicodeEncodeError
         │              └── UnicodeTranslateError
         └── Warning
                 ├── BytesWarning
                 ├── DeprecationWarning
                 ├── EncodingWarning
                 ├── FutureWarning
                 ├── ImportWarning
                 ├── PendingDeprecationWarning
                 ├── ResourceWarning
                 ├── RuntimeWarning
                 ├── SyntaxWarning
                 ├── UnicodeWarning
                 └── UserWarning
```